#### Để có thể crawl data thành các file excel có thể chọn loại data:
1. Covid By Time
2. Covid By Day
3. Recovered By Time
4. Recovered By Day


In [1]:
DataSection = ['case_by_time', 'case_by_day', 'recovered_by_time', 'recovered_by_day']
CASE_BY_TIME = 0
CASE_BY_DAY = 1
RECOVERED_BY_TIME = 2
RECOVERED_BY_DAY = 3

In [2]:
DataFile = DataSection[RECOVERED_BY_TIME]

### GET JSON File

In [3]:
import requests
import json

response = requests.get('https://covid19.ncsc.gov.vn/api/v3/covid/provinces?filter_type=' + DataFile)
decoded_response = response.json()
data = decoded_response

### GET List of Provinces

In [4]:
provResponse = requests.get('https://covid19.ncsc.gov.vn/api/v3/covid/provinces?')
provDecoded_response = provResponse.json()
proData = provDecoded_response
tempProvince = [[int(pro["id"]), pro["name"]] for pro in proData]

In [5]:
def provSort():
  for i in range(len(tempProvince)):
    for j in range(i, len(tempProvince)):
      if tempProvince[i][0] > tempProvince[j][0]:
        tempProvince[i], tempProvince[j] = tempProvince[j], tempProvince[i]

In [6]:
provSort()

### GET the list of days

In [7]:
import pandas as pd

In [8]:
HaNoi = data["1"]

In [10]:
Keys = [k for k in HaNoi.keys()]

### GET detail each of provinces

In [11]:
def getDetail(i):
  global countNull
  temp = []
  DataCurr = data[str(i)] 
  DataKey = [k for k in data[str(i)]] 
  Curr = data[str(i)]["18/3/2020"]
  sumCovid = 0
  for k in Keys:
    if k in DataKey:
      if DataCurr[k]!=0:
        sumCovid = DataCurr[k]
    temp.append(sumCovid)
  return temp

In [12]:
CovidExcel = {}
HaNoi = data["1"]
CovidExcel["Ngày"] = [k for k  in HaNoi.keys()]
for i in range(1, 64, 1):
  if i < 20:
    CovidExcel[tempProvince[i - 1][1]] = getDetail(i)
  else:
    CovidExcel[tempProvince[i - 1][1]] = getDetail(i + 1)

### Save into excel file

In [13]:
df = pd.DataFrame(CovidExcel)

In [14]:
df.to_excel('../Data/' + DataFile+ '.xlsx', sheet_name='Covid', index=False)